Import Libraries

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
import seaborn as sns
import glob
import os
from collections import defaultdict

Import Data

In [2]:
data_dir = os.path.expanduser("~/Documents/Senior/Fall/Clinic/Data Cleaner/tarsalis_data_clean")
os.chdir(data_dir)
filenames = glob.glob('*.csv')
dataframes = [pd.read_csv(f) for f in filenames]

In [56]:
%matplotlib qt
# Ones with good separations: 0, 40
# Bad separations (in one plot): 1
# Bad separation (in both): 3, 5, 6!
for i in [0, 40, 1, 3, 5, 6]:
    data = dataframes[i].query("pre_rect > -0.1 and labels in ['L', 'M', 'N', 'NP']")
    data["amp"] = data["pre_rect"].rolling(100).std()
    fig, axs = plt.subplots(2)
    sns.histplot(data = data, x = "pre_rect", hue = "labels", ax = axs[0])
    sns.histplot(data = data, x = "amp", hue = "labels", ax = axs[1])
    plt.tight_layout()
    plt.show()

C:\Users\zbtra\AppData\Local\Temp\ipykernel_22448\4028168828.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["amp"] = data["pre_rect"].rolling(100).std()
C:\Users\zbtra\AppData\Local\Temp\ipykernel_22448\4028168828.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data["amp"] = data["pre_rect"].rolling(100).std()
C:\Users\zbtra\AppData\Local\Temp\ipykernel_22448\4028168828.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,

In [53]:
plt.plot(dataframes[0]["pre_rect"])

In [52]:
plt.plot(dataframes[6]["pre_rect"])

Extract Changepoints

In [11]:
changepoints = []
for df in dataframes:
    x = df["pre_rect"].values
    cp = []
    labels = df["labels"].values
    for i in range(len(labels)):
        if i == 0:
            cp.append(i)
        elif i > 0:
            if labels[i] != labels[i - 1]:
                cp.append(i)
        elif i == len(labels) - 1:
            cp.append(i)
    changepoints.append(cp)

Make Visualizations

In [197]:
label_split = defaultdict(list)
for r in range(len(changepoints[1:2])):
    r += 4
    for i in range(len(changepoints[r]) - 1):
        start = changepoints[r][i]
        end = changepoints[r][i+1]
        label = dataframes[r]["labels"][start]
        label_split[label].append(dataframes[r]["pre_rect"].values[start:end])

In [198]:
for l, v in label_split.items():
    print(l, len(v))

NP 1
J 1
K 1
L 1
M 1
Z 1


In [199]:
dataframes[4]["labels"][changepoints[4]]

0        NP
3400      J
4424      K
4944      L
17784     M
37070     Z
37083    NP
Name: labels, dtype: object

In [201]:
%matplotlib qt
fig, ax = plt.subplots()
sns.histplot(x = label_split["K"][0],  ax = ax, stat = "percent", binwidth = 0.01, kde=True, color = "red")
sns.histplot(x = label_split["L"][0],  ax = ax, stat = "percent", binwidth = 0.01, kde=True, color = "orange")
sns.histplot(x = label_split["M"][0],  ax = ax, stat = "percent", binwidth = 0.01, kde=True, color = "green")
ax.set_xlabel("Volts")
ax.plot()

[]

In [210]:
recording = 0
plt.plot(dataframes[recording]["time"], dataframes[recording]["post_rect"])
plt.plot(dataframes[recording]["time"], dataframes[recording]["pre_rect"])
plt.vlines(np.array(changepoints[recording]) // 100, ymin = -2, ymax = 10, color = "black")
plt.xlabel("Time")
plt.ylabel("Volts")
plt.show()

In [211]:
from hmmlearn import hmm

In [302]:
models = []
scores = []
data = (dataframes[0]["pre_rect"][14505:].rolling(5 * 100).mean().dropna())
for idx in range(10):  # ten different random starting states
    # define our hidden Markov model
    model = hmm.GaussianHMM(n_components=3, random_state=idx,
                           n_iter=10)
    model.fit(data.values.reshape(-1, 1))
    models.append(model)
    scores.append(model.score(data.values.reshape(-1, 1)))
    print(f'Converged: {model.monitor_.converged}\t\t'
          f'Score: {scores[-1]}')

Converged: True		Score: -62186.15614659607
Converged: True		Score: 32387.452128080848
Converged: True		Score: 32469.267367243025
Converged: True		Score: 85680.88260374426
Converged: True		Score: 31444.60861985306
Converged: True		Score: 32409.770070111088
Converged: True		Score: 32407.047874768392
Converged: True		Score: 80674.14504782156
Converged: True		Score: 32406.785204305696
Converged: True		Score: 85679.34705520312


In [303]:
model = models[np.argmax(scores)]
print(f'The best model had a score of {max(scores)} and '
      f'{model.n_components} components')

# use the Viterbi algorithm to predict the most likely sequence of states
# given the model
states = model.predict(data.values.reshape(-1, 1))

The best model had a score of 85680.88260374426 and 3 components


In [304]:
changepoints = []
for i in range(len(states) - 1):
    if states[i] != states[i + 1]:
        changepoints.append(i)

In [284]:
len(states)

70257

In [306]:
plt.vlines(changepoints, ymin=0, ymax=2, color = "black")
plt.plot(data)

In [251]:
fig, ax = plt.subplots()
ax.imshow(model.transmat_, aspect='auto', cmap='spring')
ax.set_title('Transition Matrix')
ax.set_xlabel('State To')
ax.set_ylabel('State From')

Text(0, 0.5, 'State From')

In [322]:
data1 = dataframes[0]["pre_rect"].rolling(100).max()
data2 = dataframes[0]["pre_rect"].rolling(100).min()
plt.plot(data1 - data2)
plt.plot(dataframes[0]["pre_rect"])

In [ ]:
import stats